# Listeners must have arguments "event" and "metadata" with proper types

In [1]:
from morpheus.project.domain.events.ProjectEvents.ProjectEvents import ProjectCreatedEvent
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x109add5e0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x16a0505c0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x109add5e0>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x16a0505c0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x109add5e0>> must have an arg

# Cannot register same listener twice

In [2]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


# Create projects

In [3]:
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.types.User import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.Project import CreateProjectCommand, CreateProjectCommandHandler

for i in range(10):
  create_project_command = CreateProjectCommand(
    project_id=ProjectId.new(),
    name=Name.from_str(f'Project {i}'),
    description=Description.from_str(f'Description {i}'),
    tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
    user_id=UserId.new()
  )
  CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
  print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 1485 events
"Project Created" occurred at 2024-05-15T17:53:42.470799+02:00
"Project Created" occurred at 2024-05-15T17:54:23.911969+02:00
"Project Created" occurred at 2024-05-15T17:54:23.915577+02:00
"Project Created" occurred at 2024-05-15T17:54:23.918578+02:00
"Project Created" occurred at 2024-05-15T17:54:23.920759+02:00
"Project Created" occurred at 2024-05-15T17:54:23.922801+02:00
"Project Created" occurred at 2024-05-15T17:54:23.924870+02:00
"Project Created" occurred at 2024-05-15T17:54:23.927211+02:00
"Project Created" occurred at 2024-05-15T17:54:23.929731+02:00
"Project Created" occurred at 2024-05-15T17:54:23.932228+02:00
"Project Created" occurred at 2024-05-15T17:54:23.934704+02:00
"Project Created" occurred at 2024-05-15T17:54:32.283133+02:00
"Project Created" occurred at 2024-05-15T17:55:38.175894+02:00
"Project Created" occurred at 2024-05-15T17:56:19.510356+02:00
"Project Created" occurred at 2024-05-15T17:56:19.537236+02:00
"Project Created" occurred at 2024-05

# Reproject project summaries

In [4]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries


------------------------------


Found 1485 events


Reset projection


Successfully reprojected project summaries


In [5]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 200 project summaries
Project 8478ddf7-1c43-4aee-969e-af7dbdac18bc: Rio Primero 123
Project e0166ed1-82ae-4f82-b154-9276aa81f1fa: Project 0
Project b27d541b-e183-4083-9071-8804fd5d1b25: Project 1
Project 2ffca3ac-1fcc-4bd5-ac14-215f1a3da7a0: Project 2
Project e5194429-1545-42f2-926c-08078c49cd0b: Project 3
Project c08af565-d1c8-4679-927b-5de8f4e3aa00: Project 4
Project 05c4a721-0b33-450a-85ff-1653b4864099: Project 5
Project 410442a3-a7b7-4748-8dc9-8d75ae41d0cd: Project 6
Project 25212285-cc7e-4da7-a555-7a8dc2285ae7: Project 7
Project f877bbc8-a10d-43d3-8f2f-0e05b52ad65c: Project 8
Project 5f62697c-8ad7-4749-8442-211c3e9dd64b: Project 9
Project b88a50eb-2ffc-4142-9dbf-7324ff5a0930: Project 1
Project 60610a3c-89f8-459d-aef4-8b415c51d216: Rio Primero 123
Project 2f86c914-3848-4c4d-8893-c94777901be4: Project 0
Project 15627578-1ebb-47da-9701-0a538aa892f7: Project 1
Project ec8bd73a-45c7-4d81-a338-228525e71357: Project 2
Project 9ab27c4b-9c40-4068-b466-86c0a00ec256: Project 3
Project 